<a href="https://colab.research.google.com/github/Eastonco/CS437/blob/main/H4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment #4**

Assigned: March 5, 2021

Due: March 24, 2021



---

This assignment consists of questions that require a short answer and one Python programming task. You can enter your answers and your code directly in a Colaboratory notebook and upload the **shareable** link for your notebook as your homework submission.


---

#1.

 (10 points) Consider the subset of (x,y) points shown below. These are actually a subset of the data points found in the sklearn diabetes dataset.

x | y
--- | ---
 0.08 | 233
-0.04 | 91
 0.01 | 111
-0.04 | 152
-0.03 | 120
 0.01 | 67
 0.09 | 310
-0.03 | 94
-0.06 | 183
-0.03 | 66
 0.06 | 173
-0.06 | 72
 0.00 | 49
-0.02 | 64
-0.07 | 48


For a candidate linear regressor with parameters $\Theta_0 = 75.1$ and $\Theta_1 = -0.001$, calculate the mean squared error with respect to the data points and perform one iteration of gradient descent , assuming $\alpha = 0.01$. Show all of your work.

> Mean squared Error = 8061.61506


> Theta0 = 82.1645

> Theta1 = 30.40203


> Calculations: https://drive.google.com/file/d/1k4mOvJcBzF8uv3cV-895BDpCBY5X6EHy/view?usp=sharing


---

#2.

(15 points) Explain what value is generated by the equation $h(x) = \frac{1}{1+e^{-(\Theta_0 + \Theta_1 x)}}$ in logistic regression. What steps are taken to convert the linear regression algorithm into the classification-based logistic regression algorithm?

> this equasion generates a logistic regression line that uses classificaiton rather than rather than generating a specficic values. To convert a linear regression alorithm into a classifciation based algorithim, instead of generating a y values, you determine a classification based on if the point is above or below the generated y value
---

#3.

(12 points) Regularization is introduced in class as a numeric term that can be incorporated into a loss function. This is straightforward for algorithms such as neural networks that seek to optimize a numeric loss function. Here, explain possible ways regularization can be used to fine tune other types of algorithms, specifically decision trees and naive Bayes classifiers.

> Regularization can be used to fine tune algorithms like decision trees and naive bays classifiers by helping prevent overfit which is common in these algorithms especially when you get to very high dementionalities


---

#4.

(80 points) The goal of this program is to give you detailed experience with naive Bayes classifiers as well as with text mining methods and libraries. In this program, you are tasked with writing a naive Bayes classifier to classifier phone messages as spam versus not spam (ham). To test your program, use the labeled data available from the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection. The data is also available at https://drive.google.com/file/d/1nn2baOauApGbxrOeTb4l-30Qz1prPIS3/view?usp=sharing.

You will need to read in each message and convert it to a set of features. Use the sklearn libraries to

- remove punctuation
- convert to lower case
- create a bag of words vector that stores term frequency
- filter out words from the stop list (stop_words)
- include 1-grams and 2-grams
- normalize frequences based on document length (tfidf)

Report performance of your classifier on 3-fold cross validation in terms of accuracy and macro f1 score.

Additionally, notice that the class distribution is imbalanced. There are 4,827 legitimate messages and 747 spam messages. Experiment with three alternative methods for addressing this issue and report impact of each method on performance.

- Undersample the majority class so they are balanced.
- Oversample the minority class so they are balanced.
- Weight the data points based on class imbalance.

> Code is below 

---

#5.

(10 points, part of semester project grade)

Provide a proposal of your semester project. The proposal is typically 1-2 paragraphs long and should include the goal of your project, a description of the data you will use and any necessary data cleaning / preparation steps you will need to perform, the methods you will create or apply, and how you will evaluate the results. If you are working on a multi-person project team, clearly state the role of each person on the team in contributing to the project goal.

> My semester project will be to create a knn or neural network algorithm to determine wether or not a stock will go up or down within a certain timeframe. I will be using stock data from yahoo finance or tdameritrade. Since this is all offical data, there will be little to no data cleaning or preparation. To get a baseline of the performance, I will backtest the algorithm on historical data then do live tests on current events to see how well it works on live data and on different timeframes. My strech goal is to instead of classifying the results as up/down, instead, quanitify the strength or expected move of the stock in a %

In [ ]:
# https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html
from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import stop_words
from nltk.tokenize import TreebankWordTokenizer

import numpy as np
import pandas as pd
import pprint

from google.colab import drive
drive.mount('/content/gdrive')

def readFile(file):
    messageList = []
    classifier = []
    for line in file:
        x = line.split("\t")
        classifier.append(x[0])
        messageList.append(x[1])
    return messageList, classifier

f = open('/content/gdrive/My Drive/437/SMSSpamCollection')
messages, classifiers = readFile(f)

# print first line of the first data file as a sample
"""
print('Sample file')
print("\n".join(newsgroups.data[2].split("\n")[:10]))
print("\n")
print('data', newsgroups.data[2])
"""

# convert collection of documents to matrix of string (word) counts
count_vect = CountVectorizer()

# use regular expressions to convert text to tokens
# split contractions, separate punctuation
tokenizer = TreebankWordTokenizer()
count_vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words (try with and without this)
print(stop_words.ENGLISH_STOP_WORDS)

# include 1-grams and 2-grams
count_vect.set_params(ngram_range=(1,2))

# ignore terms that appear in >50% of the documents, try with and without this
count_vect.set_params(max_df=0.5)

# ignore terms that appear in only 1 document, try with and without this
count_vect.set_params(min_df=2)

# transform text to bag of words vector using parameters
X_counts = count_vect.fit_transform(messages)

# normalize counts based on document length
# weight common words less (is, a, an, the)
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

# train a naive Bayes classifier on data, multinomial for discrete features
clf = MultinomialNB().fit(X_tfidf, classifiers)
scores = cross_val_score(clf, X_tfidf, classifiers, cv=3, \
                         scoring='accuracy')

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, classifiers, test_size = 0.33)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#y_true_labels = list(map(lambda x: newsgroups.target_names[x], y_test))
#y_pred_labels = list(map(lambda x: newsgroups.target_names[x], y_pred))

print('Number of documents:', len(classifiers), ', 3-fold accuracy:', np.mean(scores), ', macro f1:', f1_score(y_test, y_pred, average='macro') )


# confusion matrix, show first time
print('Confusion matrix:')
#pprint.pprint(newsgroups.target_names, width=200)
print(confusion_matrix(y_test, y_pred))




  



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


Mounted at /content/gdrive
frozenset({'de', 'please', 'nine', 'most', 'amount', 'hasnt', 'hers', 'but', 'out', 'beyond', 'six', 'whereafter', 'top', 'part', 'where', 'on', 'give', 'serious', 'latterly', 'must', 'made', 'before', 'see', 'someone', 'myself', 'once', 'first', 'else', 'nor', 'him', 'former', 'about', 'found', 'otherwise', 'are', 'thence', 'himself', 'will', 'thereafter', 'cant', 'ie', 'nobody', 'anything', 'sometime', 'cannot', 'everyone', 'couldnt', 'any', 'sometimes', 'those', 'ltd', 'could', 'done', 'or', 'move', 'we', 'amongst', 'nevertheless', 'put', 'name', 'has', 'below', 'under', 'toward', 'via', 'too', 'why', 'was', 'neither', 'not', 'no', 'whole', 'thru', 'detail', 'all', 'ours', 'co', 'a', 'herself', 'whither', 'somewhere', 'above', 'three', 'last', 'others', 'full', 'twelve', 'than', 'been', 'hereafter', 'between', 'anyhow', 'nowhere', 'thin', 'another', 'next', 'thereby', 'etc', 'somehow', 'its', 'afterwards', 'when', 'yours', 'meanwhile', 'thick', 'moreover',